In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch

In [2]:
from field_setup import N_nod, Beta, K_nod, L_nod, phi4
from transforms import t_get_O    
from flows.RGflows import RGflows 
from transforms import config_RG_masks
from obs import G
normal_dist = torch.distributions.Normal(loc=torch.zeros(N_nod,phi4.dim), scale=torch.ones(N_nod,phi4.dim))

In [3]:
from samplers.Metropolis import Metropolis
from samplers.NN_Metropolis import NN_Metropolis

In [22]:
from samplers.NN_Metropolis import NN_Metropolis
N_samp = 1000
M = NN_Metropolis(phi4,N_samp,RG_trained,normal_dist,m,val = lambda x: x,log_per = 1,filename="./trajs_and_corr/0.txt") 
M.run(x)

0 0 tensor(-1.4084)
0 tensor([0.0014]) tensor(-1.3851)
0 tensor([0.0080]) tensor(-1.4065)
0 tensor([4.9956e-06]) tensor(-1.4566)
1 tensor([6.2799]) tensor(-1.4956)
2 tensor([175122.7031]) tensor(-1.4596)
3 tensor([68833.2656]) tensor(-1.4525)
3 tensor([0.0001]) tensor(-1.4425)
4 tensor([2.6374]) tensor(-1.4044)
5 tensor([101.4788]) tensor(-1.3962)
6 tensor([0.1756]) tensor(-1.3517)
6 tensor([0.0068]) tensor(-1.3416)
7 tensor([1.2031]) tensor(-1.3879)
7 tensor([1.3316e-06]) tensor(-1.3906)
7 tensor([4.2199e-06]) tensor(-1.4471)
7 tensor([5.3424e-05]) tensor(-1.4860)
8 tensor([2.8990]) tensor(-1.5322)
9 tensor([109.3137]) tensor(-1.5523)
9 tensor([0.0021]) tensor(-1.5260)
9 tensor([3.3516e-05]) tensor(-1.5151)
9 tensor([0.0339]) tensor(-1.4889)
9 tensor([1.5720e-07]) tensor(-1.5106)
9 tensor([0.0002]) tensor(-1.5425)
9 tensor([2.5418e-05]) tensor(-1.5329)
9 tensor([7.3817e-09]) tensor(-1.5385)
9 tensor([0.1153]) tensor(-1.5847)
9 tensor([0.0014]) tensor(-1.5920)
9 tensor([0.0432]) tensor


KeyboardInterrupt



In [4]:
from samplers.Langevin import Langevin
N_samp=10
z=torch.normal(mean=torch.zeros(N_samp,N_nod,phi4.dim), std=torch.ones(N_samp,N_nod,phi4.dim))
L=Langevin(phi4,N_samp,eps=0.0001,N_sweep=1000000,log_per=1000)
L.run(z)

tensor(0.9958)
tensor(0.3644)
tensor(0.2707)
tensor(0.2401)
tensor(0.2247)
tensor(0.2180)
tensor(0.2198)
tensor(0.2127)
tensor(0.2179)
tensor(0.2095)
tensor(0.2095)
tensor(0.2060)
tensor(0.1976)
tensor(0.1933)
tensor(0.1940)
tensor(0.1939)
tensor(0.1872)
tensor(0.1878)
tensor(0.1797)
tensor(0.1875)
tensor(0.1842)
tensor(0.1868)
tensor(0.1760)
tensor(0.1833)
tensor(0.1867)
tensor(0.1908)
tensor(0.1871)
tensor(0.1842)
tensor(0.1794)
tensor(0.1914)
tensor(0.1824)
tensor(0.1763)
tensor(0.1805)
tensor(0.1741)
tensor(0.1769)
tensor(0.1821)
tensor(0.1902)
tensor(0.1942)
tensor(0.1892)
tensor(0.1928)
tensor(0.1945)
tensor(0.1939)
tensor(0.1984)
tensor(0.1985)
tensor(0.2055)
tensor(0.1981)
tensor(0.1962)
tensor(0.1947)
tensor(0.1887)
tensor(0.1887)
tensor(0.1937)
tensor(0.1926)
tensor(0.1939)
tensor(0.1993)
tensor(0.1999)
tensor(0.2032)
tensor(0.2092)
tensor(0.2073)
tensor(0.2093)
tensor(0.2105)
tensor(0.2197)
tensor(0.2182)
tensor(0.2176)
tensor(0.2128)
tensor(0.2141)
tensor(0.2142)
tensor(0.2


KeyboardInterrupt

